Tracking Objects in Video with Particle Filters
===============================================

Import libraries

In [14]:
import numpy as np
import cv2

# Repeatability
np.random.seed(0)

VFILENAME = "walking.mp4"
ESC_KEY = 27

In [15]:
def get_video_data(filename):
    video = cv2.VideoCapture(filename)
    fps = round(video.get(cv2.CAP_PROP_FPS))
    framerate = round(1000 / fps) # 1000 ms / FPS
    resolution = (round(video.get(cv2.CAP_PROP_FRAME_WIDTH)),round(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    video.release()
    print(framerate)
    print(fps)
    print(resolution)
    video.release()
    return framerate, fps, resolution

In [16]:
FRAMERATE, FPS, RESOLUTION = get_video_data(VFILENAME)
WIDTH, HEIGHT = RESOLUTION

33
30
(722, 406)


Load video frames from file

In [17]:
def get_frames(filename):
    video = cv2.VideoCapture(filename) # Open Video
    while video.isOpened():
        retrieved, frame = video.read() # Read frames
        if retrieved:
            yield frame 
        else:
            break
    video.release() # Video is closed, release resources
    yield None

Creating a particle cloud

In [18]:
NUM_PARTICLES = 5000
VEL_RANGE = 0.5
# particles = [particle][(x,y)][velocityX][velocityY]
def initialize_particles():
    particles = np.random.rand(NUM_PARTICLES,4) # 5000 Rows, 4 Columns, Number between 0 and 1
    particles = particles * np.array((WIDTH,HEIGHT,VEL_RANGE,VEL_RANGE)) # Range of X, Y, and velocities for every particle
    particles[:,2:4] -= VEL_RANGE/2.0 # Center Velocity at 0
    print(particles[:20,:])
    return particles

Moving particles according to their velocity state

In [19]:
def apply_velocity(particles):
    particles[:,0] += particles[:,2]
    particles[:,1] += particles[:,3]
    return particles

Prevent particles from falling off the edge of the video frame

In [20]:
def enforce_edges(particles):
    for i in range(NUM_PARTICLES):
        particles[i,0] = max(0,min(WIDTH - 1, particles[i,0])) # Prevents x value from being greater than WIDTH
        particles[i,1] = max(0,min(HEIGHT -1, particles[i,1])) # Prevents y value from being greater than HEIGHT 
    return particles

Measure each particle's quality

In [21]:
TARGET_COLOR = np.array((156,74,38)) # BGR Color Value
def compute_errors(particles, frame):
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        x = int(particles[i,0])
        y = int(particles[i,1])
        pixel_color = frame[y,x,:] # y,x, BGR
        errors[i] = np.sum((TARGET_COLOR - pixel_color) ** 2) # Difference in color values
    return errors

Assign weights to the particles based on their quality of match

In [32]:
def compute_weights(errors):
    weights = np.max(errors) - errors # Invert errors (Highest error = Lowest Weight)
    # Set edge particles weight to 0
    weights[
        (particles[:,0] == 0) |
        (particles[:,0] == WIDTH-1) | 
        (particles[:,1] == 0) |
        (particles[:,1] == HEIGHT - 1)
    ] = 0.0
    weights = weights**4
    return weights

Resample particles according to their weights

In [33]:
def resample(particles, weights):
    probabilities = weights / np.sum(weights) # Normalize
    
    # New particles based on weights (High Weight = High Chance)
    index_numbers = np.random.choice(
        NUM_PARTICLES,
        size = NUM_PARTICLES,
        p=probabilities)
    particles = particles[index_numbers,:] # Replace particles with chosen indices
    
    x = np.mean(particles[:,0])
    y = np.mean(particles[:,1])
    return particles, (int(x),int(y))

Fuzz the particles

In [34]:
POS_SIGMA = 1.0
VEL_SIGMA = 0.5
def apply_noise(particles):
    noise = np.concatenate(
    (
        np.random.normal(0.0,POS_SIGMA,(NUM_PARTICLES,1)), # X-position
        np.random.normal(0.0,POS_SIGMA,(NUM_PARTICLES,1)), # Y-position
        np.random.normal(0.0,VEL_SIGMA,(NUM_PARTICLES,1)), # X-Velocity
        np.random.normal(0.0,VEL_SIGMA,(NUM_PARTICLES,1)), # Y-Velocity
    ),
    axis=1)
    particles += noise
    return particles

Display the video frames

In [30]:
def display(frame, particles, location):
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            x = int(particles[i,0])
            y = int(particles[i,1])
            cv2.circle(frame,(x,y),1,(0,255,0), 1)
    if len(location) > 0:
        cv2.circle(frame, location, 15, (0,0,255),5)
    cv2.imshow('frame',frame)
    if cv2.waitKey(30) == ESC_KEY: # Pause Video
        if cv2.waitKey(0) == ESC_KEY: # Exit Video
            return True
            
    return False

Main routine

In [35]:
particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, location)
    if terminate:
        break
cv2.destroyAllWindows()


[[ 2.83811719e+02  3.18830683e+02 -7.14733855e-02  1.72408515e-01]
 [ 4.05539628e+02  3.49852534e+02 -1.58670858e-01  8.88875671e-02]
 [ 3.95502071e+02  1.36817941e+02  3.15216219e-02  5.18219628e-02]
 [ 3.44595154e+02  3.88256701e+02 -1.81590797e-01  5.96008212e-02]
 [ 2.76687679e+02  4.04856492e+02 -1.20818504e-01 -2.07042298e-01]
 [ 3.67866953e+02  2.00636732e+02  1.77888302e-01 -6.00953341e-02]
 [ 4.08335903e+02  2.45082171e+02  9.12160540e-02  1.06212529e-01]
 [ 7.16005265e+02  3.74011855e+02  1.92924004e-01  1.14354502e-01]
 [ 3.74996412e+01  7.46165651e+01  2.10493735e-01 -3.67750911e-02]
 [ 2.55610849e+02  3.72525964e+02  1.16258481e-01  2.27629605e-01]
 [ 5.98722137e+02  3.28294575e+02  2.14250935e-01 -8.81296353e-02]
 [ 3.83177361e+01  1.38462024e+02 -2.34397164e-01  2.46852320e-01]
 [ 5.40487615e+02  3.48141298e+01 -6.15295968e-02  2.51379416e-02]
 [ 4.27759117e+01  3.85177267e+02 -1.25730988e-02  8.47530247e-02]
 [ 6.63288794e+01  9.56993836e+01 -5.90288316e-02 -1.02257115e

In [37]:
fourcc = cv2.VideoWriter_fourcc('M','P','4','V') # MP4V Format Output
video_out = cv2.VideoWriter("walking-output.mp4",fourcc,FPS,RESOLUTION) # (filename,fourcc,frames per sec, resolution)

def save_video(frame, particles, location):
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            x = int(particles[i,0])
            y = int(particles[i,1])
            cv2.circle(frame,(x,y),1,(0,255,0), 1)
    if len(location) > 0:
        cv2.circle(frame, location, 15, (0,0,255),5)
    video_out.write(frame)
            
    return False

In [38]:
# Save Video to Disk

particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: 
        break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    save_video(frame, particles, location)
    
cv2.destroyAllWindows()
video_out.release()


[[ 4.55815887e+02  4.76250946e+01 -1.65531380e-01 -8.74547999e-02]
 [ 6.67228944e+02  2.68577618e+02 -1.24224750e-01 -1.03997881e-01]
 [ 5.00309573e+02  3.41731035e+02 -1.71681690e-01 -1.04055360e-01]
 [ 3.37444089e+01  1.33747753e+02  1.86807285e-01 -4.01106065e-02]
 [ 9.99721733e+01  3.40452689e+02  1.19415693e-01 -1.61637885e-01]
 [ 7.47061768e+01  1.57741538e+02  1.02418736e-01  6.50624641e-02]
 [ 1.30847539e+02  9.01744547e+01 -1.24459869e-01  1.05070103e-01]
 [ 3.75729391e+02  1.17328485e+02  2.23116633e-01 -1.40917943e-01]
 [ 4.73052068e+02  5.39070101e+01  2.14857747e-01 -5.86551156e-02]
 [ 6.11703009e+02  1.64897849e+01  3.36504530e-02 -2.19661336e-01]
 [ 6.29420972e+02  7.09614847e+00  6.89989284e-02  1.08575627e-01]
 [ 2.28493306e+02  3.87315662e+02  3.45267755e-02  2.36091270e-01]
 [ 2.51318203e+02  1.97803423e+02  1.15127965e-03 -2.30458202e-01]
 [ 4.48257487e+02  2.35819284e+02 -8.39176909e-02  2.57438876e-02]
 [ 6.34409151e+02  1.85022577e+00  4.63220153e-02  2.00335252e